In [1]:
import pandas as pd

In [61]:
df = pd.read_excel("All Donation _ Data Download_ 2 July 2025, 15_23 _ Filter_ Date range 2025-06-01 s.d 2025-06-15 - waiting.xlsx")

In [62]:
def is_random_number(num):
    num = str(num)

    # Nomor harus hanya angka
    if not num.isdigit():
        return False

    # Harus valid dari segi panjang dan awalan
    starts_valid = num.startswith("08") or num.startswith("8") or num.startswith("62")
    length_valid = 8 < len(num) <= 14

    if not (starts_valid and length_valid):
        return False

    # Cek jika terlalu banyak angka sama (seperti 08888888888)
    if len(set(num)) <= 3:
        return False

    # Cek pola berulang (seperti 081234567812345678)
    if len(num) > 6 and num[:int(len(num)/2)] == num[int(len(num)/2):]:
        return False

    # Cek pola urutan (seperti 08123456789)
    if num[2:].isdigit() and num[2:] in '1234567890'*2:
        return False

    return True

df['is_random'] = df['Whatsapp'].apply(is_random_number)
df['kategori_nomor'] = df['is_random'].map({True: 'Nomor Acak', False: 'Nomor Pola/Tidak Valid'})

In [64]:
df = df[df['kategori_nomor'] == 'Nomor Acak']

In [66]:
# Standarisasi nomor agar diawali dengan 62
def format_nomor(nomor):
    nomor = str(nomor)
    if nomor.startswith('6'):
        return nomor
    elif nomor.startswith('8'):
        return '62' + nomor
    else:
        return nomor  # Bisa disesuaikan jika ada kondisi lain
    
df['Whatsapp'] = df['Whatsapp'].apply(format_nomor)

C:\Users\faruq\AppData\Local\Temp\ipykernel_13220\2940160678.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Whatsapp'] = df['Whatsapp'].apply(format_nomor)


In [67]:
agg = df.groupby("Whatsapp").agg(
    Donatur=('Donatur','first'),
    Sapaan=('Sapaan','first'),
    Program=('Program', "first"),
    Date=('Date', 'first'),
    Total=("Total", "sum")
).reset_index().sort_values(by='Total', ascending=False)

In [68]:
df = pd.read_excel('../untuk_cs/Tracking_Data/Database.xlsx')

In [69]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 13524 entries, 0 to 13523
Data columns (total 15 columns):
 #   Column               Non-Null Count  Dtype 
---  ------               --------------  ----- 
 0   Whatsapp             13524 non-null  int64 
 1   Donatur              13524 non-null  object
 2   Bulan                13524 non-null  object
 3   Tahun                13524 non-null  int64 
 4   CRM                  7934 non-null   object
 5   Day_Mode             13524 non-null  object
 6   Date_Classification  13524 non-null  object
 7   Frekuensi            13524 non-null  int64 
 8   Preferensi           13524 non-null  int64 
 9   klasifikasi_program  13524 non-null  object
 10  Rata - rata          13524 non-null  int64 
 11  Kategori             13524 non-null  object
 12  Badge                13524 non-null  object
 13  Avg Kategori         13524 non-null  object
 14  Total                13524 non-null  int64 
dtypes: int64(6), object(9)
memory usage: 1.5+ MB


In [70]:
agg['Whatsapp'] = agg['Whatsapp'].astype(int)

In [71]:
df_merge = pd.merge(agg,df, how='left', on='Whatsapp')

In [72]:
df_merge.to_excel('test.xlsx')

In [73]:
only_waiting = df_merge[df_merge['Total_y'].isna()]

In [74]:
waiting_final = only_waiting[['Whatsapp','Donatur_x','Sapaan','Program','Date','Total_x']]
waiting_final

,Whatsapp,Donatur_x,Sapaan,Program,Date,Total_x
1,6281929410976,Sunardi,Bapak,INFAK PALESTINA: BENTUK PEDULI KITA KEPADA SAU...,"Jun 06, 2025 - 02:52",1003085
2,6281371523411,Tommy Atmaja,Bapak,INFAK PALESTINA: BENTUK PEDULI KITA KEPADA SAU...,"Jun 09, 2025 - 12:33",1002795
3,6282391115299,Muliadi Manda,Bapak,INFAK PALESTINA: BENTUK PEDULI KITA KEPADA SAU...,"Jun 05, 2025 - 10:07",1000541
4,6281361691421,PEBRIADI FAHREZI,Bapak,INFAK PALESTINA: BENTUK PEDULI KITA KEPADA SAU...,"Jun 01, 2025 - 01:14",1000189
5,628586045588,Ridho ps,Bapak,SEDEKAH PALESTINA,"Jun 15, 2025 - 23:10",1000161
...,...,...,...,...,...,...
1129,6281367004569,Hamba Allah,Bapak,SEDEKAH PALESTINA,"Jun 09, 2025 - 07:02",50051
1130,6282255295414,Irma,Ibu,INFAK PALESTINA: BENTUK PEDULI KITA KEPADA SAU...,"Jun 01, 2025 - 10:47",50048
1132,6285769957389,Wahyudi,Bapak,INFAK PALESTINA: BENTUK PEDULI KITA KEPADA SAU...,"Jun 12, 2025 - 07:13",50021
1133,6281386610890,Alm Yacub A dan almh Maryati Y,Bapak,SEDEKAH PALESTINA,"Jun 04, 2025 - 06:06",50015


In [75]:
waiting_final['Bulan_Angka'] = pd.to_datetime(waiting_final['Date']).dt.month
waiting_final['Nama_Bulan'] = pd.to_datetime(waiting_final['Date']).dt.strftime('%B')
waiting_final['Nama_Hari'] = pd.to_datetime(waiting_final['Date']).dt.strftime('%A')
waiting_final['Date'] = pd.to_datetime(waiting_final['Date']).dt.date

C:\Users\faruq\AppData\Local\Temp\ipykernel_13220\1233219089.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  waiting_final['Bulan_Angka'] = pd.to_datetime(waiting_final['Date']).dt.month
C:\Users\faruq\AppData\Local\Temp\ipykernel_13220\1233219089.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  waiting_final['Nama_Bulan'] = pd.to_datetime(waiting_final['Date']).dt.strftime('%B')
C:\Users\faruq\AppData\Local\Temp\ipykernel_13220\1233219089.py:3: SettingWithCopyWarning: 
A value is trying to be set on

In [77]:
def check_date(df):
    if df['Nama_Hari'] == 'Friday':
        return 'Jumat'
    elif (df['Date'].day >= 26 or df['Date'].day <= 5):
        return 'Masa Gajian'
    else:
        return 'Bukan Masa Gajian'
    

waiting_final['Status'] = waiting_final.apply(check_date, axis=1)

C:\Users\faruq\AppData\Local\Temp\ipykernel_13220\930156002.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  waiting_final['Status'] = waiting_final.apply(check_date, axis=1)


In [79]:
waiting_final.to_excel("mas_mizan.xlsx")